<a href="https://colab.research.google.com/github/martinj2-dot/CSC386-Wall-Follower/blob/main/username_CSC386_Wall_Follower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A00: Wall Follower
* This assignment should be done individually or with a partner.
* You should seek help completing this assignment from the TAs at the evening lab.

### Learning Objectives 
* Practice breaking a larger problem down into smaller pieces using functions.
* Gain practice manipulating arrays.
* Establish and understanding of the Wall-Follower algorithm and its purpose

### How to Start
* To begin, save a copy in Drive and share it with all members of your team.
* Change the file name of this document to username1, username2 - A00: Wall Follower (for example, neillz, martinj2 - A00: Wall Follower). 

### Additional Notes
This assignment utilizes Google Colab. If you are unfamiliar with Colab, don’t worry. The idea is that you can write both code and read text. **Real-time saving was deprecated in 2018. Your partner must refresh the page to see your changes.**

The assignment has three activities: Drive Square, Drive Safe, and Wall Follower. There is no git workflow associated with this project. Once you are finished, submit the link to Moodle.

In [ ]:
HIGH = 1e6

class Node: 
  """
  parent: The index of the parent node of the current node
  distances: the distance of each node, inatilized high and updated from data in mi_graph.txt
  visited: boolean type, whether or not the node has been visited
  queued: boolean type, whether or not the node has been queued
  """
  def __init__(self):
    self.parent = -1
    self.distances = 50000
    self.visited = False
    self.queued = False

# Functions we will be using

def nameToIdx(name, v):
  for i in range(len(v)):
    if v[i] == name: return i 
    else: return -1

def createGraph(file_path):
  g = Graph()
  ifstream in(file_path)
  if not in.is_open():
    f"ERROR: Failed to load graph from {file_path}"
    return g

  s = ""
  N = 0
  while s != "NODES": in >> s >> N

  for i in range(N):
    in >> s
    g.data.push_back(s)

  g.edges = []
  g.edge_costs = []

  while s != "EDGES": in >> s >> N

  city1, city2 = ""
  for i in range(N):
    in >> city1 >> city2 >> dist
    c1 = nameToIdx(city1, g.data)
    c2 = nameToIdx(city2, g.data)
    g.edges[c1].push_back(c2)
    g.edges[c2].push_back(c1)
    g.edge_costs[c1].push_back(dist)
    g.edge_costs[c2].push_back(dist)
  return g

def tracePath(n, g):
  path = []
  curr = n
  while curr != -1:
    path.push_back(curr)
    curr = getParent(curr, g)

  # Since we built the path backwards, we need to reverse it.
  reverse(path.begin(), path.end())
  return path

def printPath(path, g):
  if len(path) < 1:
    print("No path found :(\n")
    return 0

  print("Path: ")
    for i in range(len(path) -1) f"{g.data[path[i]]} -> ")
    print(g.data[path.back()] + "\n")

def getNeighbors(n, g): return g.edges[n]

def getEdgeCosts(n, g): return g.edge_costs[n]

def getParent(idx, g): return g.nodes[idx].parent 

def initGraph(g):
  # populates a vector of nodes with size(graph data)
    for i in range(len(g.data)):
      node = Node()
      g.nodes.push_back(node)
    }
}

def bfs(start, goal, g):
  initGraph(g)
  path = []
  visit_list = []
  currentIdx = start
  g.nodes[currentIdx].distances = 0
  g.nodes[currentIdx].queued = True
  visit_list.push(currentIdx)

  while not visit_list.empty():
    currentIdx = visit_list.front()
    g.nodes[currentIdx].visited = true
    neighbors = getNeighbors(currentIdx, g)
    edge_weights = getEdgeCosts(currentIdx, g)
    if currentIdx == goal:           # Checks to see if current node is goal node. Returns path of it is
      path = tracePath(currentIdx, g)
      break
    visit_list.pop()

    for i in range(len(neighbors)):                # Populates node data for each node, within the vector of nodes.
      if not g.nodes[neighbors[i]].visited:        # Checks to see if the node has been visited
        if not g.nodes[neighbors[i]].queued:       # Checks to see if the node has been queued
          visit_list.push(neighbors[i])            # Pushes current node to the back of the queue if all conditions are false

        if g.nodes[neighbors[i]].distances > g.nodes[currentIdx].distances + edge_weights[i]: # what happens if we run into a loop
          g.nodes[neighbors[i]].distances = g.nodes[currentIdx].distances + edge_weights[i] # updates distances of current node
          g.nodes[neighbors[i]].parent = currentIdx # sets parent index to current index of the current node
  return path

def main() {
  Graph g = createGraph("graph.txt")

  start = nameToIdx("ann_arbor", g.data)
  goal = nameToIdx("marquette", g.data)

  f"Searching for a path from {g.data[start]} (index: {start}) to {g.data[goal]} (index: {goal})...\n"
  path = bfs(start, goal, g)
  printPath(path, g)

  return 0

SyntaxError: ignored

### Submission Instructions
* Set the link to “Anyone with the link can view”.
* Submit the link to Moodle before the date: **MM/DD/YY**